In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class SVM:
    def __init__(self, X, y, kernel_type="linear", max_iter=10000, C=1.0, tolerance=0.001):
        # parameters
        self.kernels = {
            'linear': self.kernel_linear,
            'quadratic': self.kernel_quadratic
        }
        self.kernel_type = kernel_type

        self.max_passes = max_iter # max passes
        self.C = C # regularization paramtere
        self.tol = tolerance # tolerance

        # input/training-data
        self.X = X
        self.N, self.D = self.X.shape
        self.y = y


    def fit(self):
        alpha = np.zeros((self.N, ))
        b = 0
        passes = 0
        kernel = self.kernels[self.kernel_type]
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(self.N):
                Ei = self.get_Ek(i, self.get_w(alpha), b)
                if(((self.y[i]*Ei) < -self.tol and alpha[i] < self.C) or ((self.y[i]*Ei) > self.tol and alpha[i] > 0)):
                    j = self.get_rnd_int(self.N-1, i)  # Get random int i~=j
                    Ej = self.get_Ek(j, self.get_w(alpha), b)

                    alpha_old = np.copy(alpha) # save old alphas

                    L,H = self.get_L_H(alpha[j], alpha[i], self.y[j], self.y[i])

                    if(L == H):
                        continue
                    eta = -kernel(self.X[i], self.X[i]) -kernel(self.X[j], self.X[j]) + 2 * kernel(self.X[i], self.X[j])
                    if(eta>=0):
                        continue
                    alpha[j] = alpha[j] - self.y[j]*((Ei - Ej)/eta)
                    alpha[j] = max(alpha[j], L)
                    alpha[j] = min(alpha[j], H)
                    if(abs(alpha[j] - alpha_old[j]) < 1e-5):
                        continue

                    alpha[i] = alpha_old[i] + self.y[i]*self.y[j] * (alpha_old[j] - alpha[j])

                    b1 = b - Ei - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[i].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[i], self.X[j].T)
                    b2 = b - Ej - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[j].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[j], self.X[j].T)
                    if((0 < alpha[i] and alpha[i] < self.C) and (0 < alpha[j] and alpha[j] < self.C)):
                        b = (b1 + b2)/2
                    elif((0 < alpha[i] and alpha[i] < self.C)):
                        b = b1
                    elif ((0 < alpha[j] and alpha[j] < self.C)):
                        b = b2

                    num_changed_alphas+=1

            if(num_changed_alphas == 0):
                passes+=1
            else:
                passes = 0

        self.alpha = alpha
        self.w = self.get_w(alpha)
        self.b = b

        # print(passes)

        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = self.X[alpha_idx, :]
        return support_vectors, passes

    def predict(self, X):
        # kernel = self.kernels[self.kernel_type]
        # eval = np.sign(np.dot(np.multiply(self.alpha, self.y), kernel(X, X)) + self.b).astype(int)

        return np.sign(np.dot(self.w, X.T) + self.b).astype(int)

    def get_w(self, alpha):
        return np.dot(np.multiply(alpha, self.y), self.X)

    def get_rnd_int(self, n, z):
        # TODO: If does not work, use external method
        arr = np.arange(n)
        np.random.shuffle(arr)
        if arr[0] == z:
            return arr[1]
        else:
            return arr[0]

    def f(self, i, w, b):
        return np.sign(np.dot(w.T, self.X[i].T) + b).astype(int)

    def get_Ek(self, i, w, b):
        # print(np.dot(w.T, self.X[i].T) + b, self.y[i], self.X[i])
        return self.f(i,  w, b) - self.y[i]

    def get_L_H(self, alpha_j, alpha_i, y_j, y_i):
        if (y_i != y_j):
            return (max(0, alpha_j - alpha_i), min(self.C, self.C - alpha_i + alpha_j))
        else:
            return (max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j))
#
    #  Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [4]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [5]:
class_chosen = 1 # only this class is chosen
y = np.asarray([-1 if y[i]!=class_chosen else 1 for i in range(y.shape[0])])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# print("Support vector count: %d" % (sv_count))
# print("bias:\t\t%.3f" % (model.b))
# print("w:\t\t" + str(model.w))
# print("accuracy:\t%.3f" % (acc))
# print("Converged after %d iterations" % (iterations))

In [8]:
from skimage.feature import hog
from skimage import data, exposure
import cv2
def load_images(folder):
#     images = [np.array([])]
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(20,20))
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
#         hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        if img is not None:
            hog_image = hog_image.flatten()
#             images= np.append(images,hog_image_rescaled)
            images.append(hog_image)
            output = np.vstack(images)
#             images=np.vstack([hog_image_rescaled, hog_image_rescaled])
    return output

In [9]:
import os
imgdata = load_images('../../../data')

In [10]:
imgdata.shape

(50, 400)

In [11]:
labels = np.array([])
for i in range(25):
    labels = np.append(labels,1)

In [12]:
for i in range(25):
    labels = np.append(labels,-1)

In [13]:
labels.shape

(50,)

In [14]:
C=1.0
epsilon=0.0001
model = SVM(imgdata, labels, C=C, tolerance=epsilon)

In [15]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [16]:
support_vectors, iterations = model.fit()

In [17]:
sv_count = support_vectors.shape[0]

In [18]:
alien1 = cv2.imread('../../../data/alien1.jpg')
alien1 = cv2.resize(alien1,(20,20))
fd, alien1 = hog(alien1, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
alien1 = alien1.flatten()
alien1.shape

(400,)

In [19]:
predator1 = cv2.imread('../../../data/predator2.jpg')
predator1 = cv2.resize(predator1,(20,20))
fd, predator1 = hog(predator1, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
predator1 = predator1.flatten()
predator1.shape

(400,)

In [20]:
pred = model.predict(predator1)
pred

-1

In [21]:
def load_predict(folder):
    predictions = np.array([])
    w= np.array([])
    b= np.array([])
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(20,20))
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
        if img is not None:
            predicted = hog_image.flatten()
            prediction = model.predict(predicted)
#             print(clf.w, clf.b, prediction)
#             predictions = np.array(prediction)
            predictions = np.append(predictions,prediction)
    return predictions

In [22]:
alienpredic = load_predict('../../dataset/image/validation/alien')

In [23]:
alienpredic

array([-1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,
        1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,
        1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,
        1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,
       -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,
       -1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,
       -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.])

In [24]:
preadtorpredic = load_predict('../../dataset/image/validation/predator')

In [25]:
preadtorpredic

array([ 1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,
        1.,  1., -1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,
       -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,
       -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1., -1., -1.,
       -1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
        1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,
       -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.])

In [26]:
num = 0
for i,val in enumerate(alienpredic.astype(int)):
    if val == 1:
        num += 1
        print(num, "ALIEN")

1 ALIEN
2 ALIEN
3 ALIEN
4 ALIEN
5 ALIEN
6 ALIEN
7 ALIEN
8 ALIEN
9 ALIEN
10 ALIEN
11 ALIEN
12 ALIEN
13 ALIEN
14 ALIEN
15 ALIEN
16 ALIEN
17 ALIEN
18 ALIEN
19 ALIEN
20 ALIEN
21 ALIEN
22 ALIEN
23 ALIEN
24 ALIEN
25 ALIEN
26 ALIEN
27 ALIEN
28 ALIEN
29 ALIEN
30 ALIEN
31 ALIEN
32 ALIEN
33 ALIEN
34 ALIEN
35 ALIEN
36 ALIEN
37 ALIEN
38 ALIEN
39 ALIEN
40 ALIEN
41 ALIEN
42 ALIEN
43 ALIEN


In [27]:
num = 0
for i,val in enumerate(preadtorpredic.astype(int)):
    if val == -1:
        num += 1
        print(num, "PREDATOR")

1 PREDATOR
2 PREDATOR
3 PREDATOR
4 PREDATOR
5 PREDATOR
6 PREDATOR
7 PREDATOR
8 PREDATOR
9 PREDATOR
10 PREDATOR
11 PREDATOR
12 PREDATOR
13 PREDATOR
14 PREDATOR
15 PREDATOR
16 PREDATOR
17 PREDATOR
18 PREDATOR
19 PREDATOR
20 PREDATOR
21 PREDATOR
22 PREDATOR
23 PREDATOR
24 PREDATOR
25 PREDATOR
26 PREDATOR
27 PREDATOR
28 PREDATOR
29 PREDATOR
30 PREDATOR
31 PREDATOR
32 PREDATOR
33 PREDATOR
34 PREDATOR
35 PREDATOR
36 PREDATOR
37 PREDATOR
38 PREDATOR
39 PREDATOR
40 PREDATOR
41 PREDATOR
42 PREDATOR
43 PREDATOR
44 PREDATOR
45 PREDATOR
46 PREDATOR
47 PREDATOR
48 PREDATOR
49 PREDATOR
50 PREDATOR
51 PREDATOR
52 PREDATOR
53 PREDATOR
54 PREDATOR
